In [9]:
from tqdm import tqdm 

import os

import numpy as np

import torch
import torch.nn as nn

from torch.optim import Adam

from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor, Normalize, Compose
from torch.utils.data import DataLoader
from torchvision.utils import save_image, make_grid

from unet import Unet
from scheduler import get_schedules

In [10]:
# Hyper parameters

epochs = 20  # If training takes too long you can reduce the number of epochs or increase the batch size
batch_size = 64
lr = 1e-5

n_T = 1000
betas = [1e-4, 0.02]

In [11]:
torch.manual_seed(42)

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [12]:
# Import dataset
transform = Compose([ToTensor(), Normalize((0.5,), (0.5,))])
dataset = MNIST("./data", train=True, download=True, transform=transform,
)

dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=2)

In [13]:
# Initialize model, loss function, and optimizer
unet = Unet(n_features=128).to(device)
loss_fn = nn.MSELoss()
optim = Adam(unet.parameters(), lr=lr)

# pre-compute schedules
schedules = get_schedules(betas[0], betas[1], n_T)
schedules = {key: val.to(device) for key, val in schedules.items()}  # add all tensors on device

In [14]:
# Create results directory if it does not exist
if not os.path.isdir('results'):
    os.makedirs('results')

if not os.path.isdir('saved_models'):
    os.makedirs('saved_models')

In [15]:
def sample(model, n_T, n_samples, sample_shape, device, schedules):

    # Step 1
    x_T = torch.randn(n_samples, *sample_shape).to(device)
    ones = torch.ones(n_samples).to(device)

    # Step 2
    x_i = x_T
    for i in tqdm(range(n_T, 0, -1)):
        # Step 3
        z = torch.randn(n_samples, *sample_shape).to(device) if i > 1 else 0
        # Step 4
        t = (i / n_T)*ones
        eps = model(x_i, t)
        eps = eps.clone().detach()
        v = schedules["one_over_sqrt_a"][i] * (x_i - eps * schedules["inv_alpha_over_sqrt_inv_abar"][i])
        x_i = v + schedules["sqrt_beta"][i] * z


    # Step 6
    x = x_i
    return x

In [16]:
# Training loop
losses = []
for epoch in range(epochs):
    print(f"Epoch {epoch+1} : ")

    # Set unet in training mode
    unet.train()

    pbar = tqdm(dataloader)
    for x, _ in pbar:
        x = x.to(device)
        x = x.view(-1, 1, 28, 28)

        # Step 3
        timesteps = torch.randint(1, n_T + 1, (x.shape[0],)).to(device)

        # Step 4
        eps = torch.randn_like(x)

        # Step 5
        optim.zero_grad()

        x_t = schedules["sqrt_abar"][timesteps, None, None, None] * x + schedules["sqrt_inv_abar"][timesteps, None, None, None] * eps
        t = timesteps/n_T
        eps_hat = unet(x_t, t)


        loss = loss_fn(eps_hat, eps)
        loss.backward()
        losses.append(loss.item())
        avg_loss = np.asarray(losses)[-10:].mean()
        pbar.set_description(f"Moving average loss: {avg_loss:.4f}")
        optim.step()

    # Set unet in eval mode
    unet.eval()
    with torch.no_grad():
        x_hat = sample(unet, n_T, 8, (1, 28, 28), device, schedules)
        grid = make_grid(x_hat, normalize=True, value_range=(-1, 1), nrow=2)

        # IMPORTANT: the grid contains 
        save_image(grid, f"results/sample_mnist{epoch+1}.png")

        # save model
        torch.save(unet.state_dict(), f"saved_models/unet_mnist.pt")

Epoch 0 : 


100%|██████████| 1000/1000 [00:13<00:00, 72.94it/s]


Epoch 1 : 


100%|██████████| 1000/1000 [00:14<00:00, 69.48it/s]


Epoch 2 : 


...

Epoch 6 : 


100%|██████████| 1000/1000 [00:13<00:00, 73.40it/s]


Epoch 7 : 


Avg loss: 0.1143:  20%|█▉        | 186/938 [00:56<03:48,  3.29it/s]


KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt

plt.figure()
plt.plot(losses)
plt.xlabel('Iterations')
plt.ylabel('Loss')